In [ ]:
from confluent_kafka import Consumer, KafkaError

# Confluent Cloud configuration
bootstrap_servers = 'pkc-9q8rv.ap-south-2.aws.confluent.cloud:9092'
api_key = 'O4GP3Y7WWONULYWH'
api_secret = 'VA0UHM3At4gnfYiX4Qz5ITkBZNBs5SBhDM5IiqiO+peKVpVU3B83KOgI8uypwNFG'
topic_inventory = 'inventory_orders'

def main():
    # Kafka consumer configuration for Confluent Cloud
    consumer_config = {
        'bootstrap.servers': bootstrap_servers,
        'security.protocol': 'SASL_SSL',
        'sasl.mechanism': 'PLAIN',
        'sasl.username': api_key,
        'sasl.password': api_secret,
        'group.id': 'inventory-consumer-group',  # Use a unique consumer group ID
        'auto.offset.reset': 'earliest'  # Start consuming from the earliest offset
    }

    # Create Kafka consumer
    consumer = Consumer(consumer_config)

    # Subscribe to the inventory orders topic
    consumer.subscribe([topic_inventory])

    try:
        while True:
            # Poll for messages
            msg = consumer.poll(timeout=1.0)

            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition, continue polling
                    continue
                else:
                    # Error occurred, handle it or raise an exception
                    print(f'Error: {msg.error()}')
                    break

            # Process the message
            message_value = msg.value().decode('utf-8')
            message_data = json.loads(message_value)

            # Check if the message type is 'inventory' and process accordingly
            if message_data['type'] == 'inventory':
                # Update inventory databases or systems
                update_inventory(message_data)

    except KeyboardInterrupt:
        # Stop consumer on keyboard interrupt
        consumer.close()

def update_inventory(data):
    # Implement your logic to update inventory databases or systems
    # This is a placeholder function, replace it with your actual logic
    print(f"Updating inventory for order_id: {data['order_id']}")

if __name__ == "__main__":
    main()
